In [14]:
import os
import numpy as np
import pandas as pd
import glob
import sys
sys.path.append("../common_tools/peaks_confusion_matrix")
from confusion_matrix import compute_confusion_matrix
from plot_false_peaks import plot_fp_fn

In [ ]:
def terma_detect_peaks(input_dir, output_dir, w_cycle=55, w_evt=9, beta=0.095):
    """
    Processes multiple PPG files, detects peaks, and saves the results into CSV files.
    
    Parameters:
    - input_dir: Path to the directory containing input PPG files.
    - output_dir: Path to the directory where the output files will be saved.
    - w_cycle: Window size for calculating the cycle moving average.
    - w_evt: Window size for calculating the event moving average.
    - beta: Scaling factor for the threshold.
    """
    
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):  
            input_file = os.path.join(input_dir, file_name)
            print(f"Processing file: {file_name}")
            
            # Read the PPG data from the file
            ppg_data = pd.read_csv(input_file, header=None)
            signal = np.asarray(ppg_data.iloc[:, 0], dtype=np.float64) 
            
            # Shift the signal to eliminate negative values
            min_signal = np.min(signal)
            if min_signal < 0:
                signal = signal - min_signal 

            # Enhance the signal
            signal = signal * signal
            
            # Initialize arrays for moving averages and thresholds
            ma_cycle = np.zeros_like(signal, dtype=np.float64)
            ma_evt = np.zeros_like(signal, dtype=np.float64)
            mean_signal = np.mean(signal)
            mean = np.full(len(signal), mean_signal)
            threshold_1 = np.zeros_like(signal, dtype=np.float64)
            block_of_interest = np.zeros_like(signal, dtype=np.float64)
            peak_index_arr = []
            
            # Calculate the Event Duration Moving Average
            for i in range((w_evt - 1) // 2, len(signal) - ((w_evt - 1) // 2)):
                ma_evt[i] = np.mean(signal[i - (w_evt - 1) // 2 : i + (w_evt - 1) // 2 + 1])
            
            # Calculate the Event Cycle Moving Average
            for i in range((w_cycle - 1) // 2, len(signal) - ((w_cycle - 1) // 2)):
                ma_cycle[i] = np.mean(signal[i - (w_cycle - 1) // 2 : i + (w_cycle - 1) // 2 + 1])
            
            # Calculate the Threshold for Block of Interest
            for i in range(len(signal)):
                threshold_1[i] = ma_cycle[i] + beta * mean[i]
            
            # Generate the Block of Interest
            for i in range(len(signal)):
                if (ma_evt[i] > threshold_1[i]) and (ma_cycle[i] != 0):
                    block_of_interest[i] = 1
                else:
                    block_of_interest[i] = 0
            
            # Peak detector logic
            start_block = 0
            stop_block = 0
            peak = 0
            peak_index = 0
            block_num = 0
            
            for i in range(len(block_of_interest) - 1):
                if (block_of_interest[i + 1] - block_of_interest[i]) == 1:
                    start_block = i
                if (block_of_interest[i] - block_of_interest[i + 1]) == 1:
                    stop_block = i
                    if (stop_block - start_block) >= w_evt:
                        peak = signal[start_block]
                        for j in range(start_block, stop_block + 1):
                            if signal[j] > peak:
                                peak = signal[j]
                                peak_index = j
                        peak_index_arr.append(peak_index)
                        block_num += 1
            
            output_file = os.path.join(output_dir, f"{file_name.replace('.csv', '')}_detected_peaks.csv")
            pd.DataFrame(peak_index_arr).to_csv(output_file, header=False, index=False)
            print(f"Peaks saved to: {output_file}")


In [ ]:
test_data_dir = 'E:/dilated_cnn_peak_detection_model_data/test/test_data/data'
label_dir = 'E:/dilated_cnn_peak_detection_model_data/test/test_data/label'
output_dir = 'E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/'
plot_output_dir = 'E:/dilated_cnn_peak_detection_model_data/test/algorithm_plot_results/'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(plot_output_dir, exist_ok=True)

for file in glob.glob(os.path.join(output_dir, "*.csv")):
    os.remove(file)

for file in glob.glob(os.path.join(plot_output_dir, "*.png")):
    os.remove(file)

In [17]:
terma_detect_peaks(test_data_dir, output_dir)

Processing file: 0009_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0009_8min_signal_detected_peaks.csv
Processing file: 0015_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0015_8min_signal_detected_peaks.csv
Processing file: 0016_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0016_8min_signal_detected_peaks.csv
Processing file: 0018_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0018_8min_signal_detected_peaks.csv
Processing file: 0023_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0023_8min_signal_detected_peaks.csv
Processing file: 0028_8min_signal.csv
Peaks saved to: E:/dilated_cnn_peak_detection_model_data/test/algorithm_detected_peaks/0028_8min_signal_detected_peaks.csv
Processing file: 0029_8min_signal.

In [18]:
# Test and save results
total_tp, total_fp, total_fn = 0, 0, 0
num_files = 0

for file_name in os.listdir(test_data_dir):
    input_file = os.path.join(test_data_dir, file_name)
    ppg_data = pd.read_csv(input_file, header=None).squeeze("columns").values

    labeled_file = os.path.join(label_dir, file_name.replace(".csv", "_labeled_peaks.csv"))
    detected_file = os.path.join(output_dir, file_name.replace(".csv", "_detected_peaks.csv"))
    
    if os.path.exists(labeled_file) and os.path.exists(detected_file):
        labeled_peaks = pd.read_csv(labeled_file, header=None).squeeze("columns").values
        detected_peaks = pd.read_csv(detected_file, header=None).squeeze("columns").values

        # Compute confusion matrix
        results = compute_confusion_matrix(detected_peaks, labeled_peaks)
        print(f"File: {file_name}")
        print(f"True Positives (TP): {results['tp']}")
        print(f"False Positives (FP): {results['fp']}")
        print(f"False Negatives (FN): {results['fn']}")
        print(f"Accuracy: {results['accuracy']:.2f}")
        print(f"Precision: {results['precision']:.2f}")
        print(f"Recall: {results['recall']:.2f}")
        print(f"Detection Error Rate: {results['der']:.2f}")
        print("False Positives (FP) at indices:", results['fp_list'])
        print("False Negatives (FN) at indices:", results['fn_list'])
        print("")

        # Accumulate statistics
        total_tp += results['tp']
        total_fp += results['fp']
        total_fn += results['fn']
        num_files += 1

        # Plot False Positives and False Negatives
        plot_fp_fn(ppg_data, results['fp_list'], results['fn_list'], detected_peaks, file_name, plot_output_dir)

# Compute overall metrics
if num_files > 0:
    overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    overall_accuracy = total_tp / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0
    overall_der = (total_fp + total_fn) / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0

    print("\n===== Overall Performance =====")
    print(f"Total Files Processed: {num_files}")
    print(f"Overall Accuracy: {overall_accuracy:.2f}")
    print(f"Overall Precision: {overall_precision:.2f}")
    print(f"Overall Recall: {overall_recall:.2f}")
    print(f"Overall Detection Error Rate (DER): {overall_der:.2f}")
else:
    print("No valid files processed.")

File: 0009_8min_signal.csv
True Positives (TP): 814
False Positives (FP): 0
False Negatives (FN): 2
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
Detection Error Rate: 0.00
False Positives (FP) at indices: []
False Negatives (FN) at indices: [20, 18810]

File: 0015_8min_signal.csv
True Positives (TP): 959
False Positives (FP): 1
False Negatives (FN): 1
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
Detection Error Rate: 0.00
False Positives (FP) at indices: [37725]
False Negatives (FN) at indices: [37717]

File: 0016_8min_signal.csv
True Positives (TP): 1000
False Positives (FP): 3
False Negatives (FN): 2
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
Detection Error Rate: 0.01
False Positives (FP) at indices: [1723, 1774, 1774]
False Negatives (FN) at indices: [1694, 1762]

File: 0018_8min_signal.csv
True Positives (TP): 1125
False Positives (FP): 0
False Negatives (FN): 1
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
Detection Error Rate: 0.00
False Positives (FP) at indices: []
False Negative